In [ ]:
import os
import json
import re
import pandas as pd
import numpy as np
import csv
import torch
import jionlp as jio
from torch.optim.lr_scheduler import LambdaLR, StepLR, MultiStepLR, ExponentialLR, ReduceLROnPlateau 

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
# load the model and tokenizer

tokenizer = AutoTokenizer.from_pretrained("trained_model/model4")
device = torch.device('cuda')
#change the model here
model = AutoModelForSeq2SeqLM.from_pretrained('trained_model/model4')

In [ ]:
def generate_answer(question):
    inputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True, max_length=256).to(device)
    out = model.generate(**inputs)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:

model = model.to(device)
question = '房東甲乙丙丁，出租未保存登記的建物出租給某藥師戊，戊於簽約時即將其妻子己列為連帶保證人，雙雙用印。豈料，藥師戊因為積欠大筆債務，遂將租賃建物一半違法轉租租給花店庚，但仍無法負擔大筆債務，於租賃期限到期前跑路。到期後房東請求藥師搬遷，豈料建物中找不到藥師，手機電話也連絡不上。房東遂起訴妻子己及次承租人庚騰空建物，妻子己作為連帶保證人應依約給付為限期遷出的違約金。'
generate_answer(question)


In [ ]:
metric = load_metric("sacrebleu")
model.eval()
predictions = []
labels = []
for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels_ids = batch['labels'].to(device)
    output = model.generate(input_ids, attention_mask=attention_mask)
    predictions.extend(tokenizer.batch_decode(output, skip_special_tokens=True))
    labels.extend(tokenizer.batch_decode(labels_ids, skip_special_tokens=True))


scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = []
for i in range(len(predictions)):
    scores.append(scorer.score(predictions[i], labels[i]))

rouge1 = [score['rouge1'].fmeasure for score in scores]
rouge2 = [score['rouge2'].fmeasure for score in scores]
rougeL = [score['rougeL'].fmeasure for score in scores]

np.mean(rouge1), np.mean(rouge2), np.mean(rougeL)